In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.applications import MobileNetV2, ResNet152, VGG16, EfficientNetB0, InceptionV3
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
import os
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader


In [5]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels


In [6]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, target_size=(236, 236))
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(features.shape[0], 236, 236, 3)  # Reshape all images in one go
    return features


In [7]:
TRAIN_DIR = "/kaggle/input/task1-data/Data/Train"


In [8]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

train_features = extract_features(train['image'])

x_train = train_features / 255.0

AI completed
Real completed


  0%|          | 0/590 [00:00<?, ?it/s]

In [9]:
le = LabelEncoder()
le.fit(train['label'])
y_train = le.transform(train['label'])
y_train = to_categorical(y_train, num_classes=2)

In [17]:
model = Sequential()
# Convolutional layers
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(236, 236, 3)))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(1024, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2048, activation='relu'))
model.add(Dense(2, activation='softmax'))
LEARNING_RATE = 0.001





model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x=x_train, y=y_train, batch_size=20, epochs=30)

Epoch 1/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 60s 1s/step - accuracy: 0.5522 - loss: 62.1339
Epoch 2/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 569ms/step - accuracy: 0.6679 - loss: 36.5326
Epoch 3/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 572ms/step - accuracy: 0.6365 - loss: 28.6691
Epoch 4/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 577ms/step - accuracy: 0.7152 - loss: 14.8445
Epoch 5/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 584ms/step - accuracy: 0.7679 - loss: 8.4151
Epoch 6/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 595ms/step - accuracy: 0.7557 - loss: 6.5271
Epoch 7/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 604ms/step - accuracy: 0.8058 - loss: 6.0265
Epoch 8/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 604ms/step - accuracy: 0.8160 - loss: 4.2900
Epoch 9/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 595ms/step - accuracy: 0.8237 - loss: 3.0840
Epoch 10/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 594ms/step - accuracy: 0.8285 - loss: 2.6288
Epoch 11/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 591ms/step - accuracy: 0.8442 - loss: 2.8520
Epoch 12/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 586ms/st

In [18]:
def fine_tune_model(base_model, layers_to_train):
    base_model.trainable = True
    for layer in base_model.layers[:-layers_to_train]:
        layer.trainable = False


In [19]:
TEST_DIR ='/kaggle/input/task1-data/Data/Test'
test_images = []
test_image_ids = []

for imagename in os.listdir(TEST_DIR):
    test_images.append(os.path.join(TEST_DIR, imagename))
    test_image_ids.append(imagename)

test_df = pd.DataFrame({"Id": test_image_ids, "image": test_images})

In [20]:
test_features = extract_features(test_df['image'])

x_test = test_features / 255.0




test_predictions = model.predict(x_test)

predicted_labels = np.argmax(test_predictions, axis=1)

label_map = {0: "AI", 1: "Real"}
test_df['Label'] = [label_map[label] for label in predicted_labels]






  0%|          | 0/200 [00:00<?, ?it/s]

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 118ms/step


In [21]:
submission_file = "submission1.csv"
test_df[['Id', 'Label']].to_csv(submission_file, index=False)

print(f"Submission file saved as {submission_file}")

Submission file saved as submission1.csv


In [23]:
saved_file = pd.read_csv(submission_file)
print(saved_file.head())

              Id Label
0   image_22.jpg    AI
1   image_53.jpg    AI
2  image_141.jpg    AI
3   image_69.jpg    AI
4   image_68.jpg  Real
